In [1]:
import pandas as pd
train=pd.read_csv('D:\\天池比赛\\基金预测\\product_info_simple_final_train.csv')
time=pd.read_csv('D:\\天池比赛\\基金预测\\time_info_final.csv')
profit_rate=pd.read_csv('D:\\天池比赛\\基金预测\\cbyieldcurve_info_final.csv')
test=pd.read_csv('D:\\天池比赛\\基金预测\\predict_table.csv')


In [2]:
unique_product_pids = test['product_pid'].unique()
print(unique_product_pids)

['product1' 'product100' 'product101' 'product102' 'product103'
 'product104' 'product105' 'product106' 'product107' 'product108'
 'product109' 'product110' 'product111' 'product112' 'product113'
 'product114' 'product115' 'product116' 'product117' 'product118'
 'product119' 'product12' 'product120' 'product121' 'product122'
 'product123' 'product124' 'product125' 'product126' 'product128'
 'product129' 'product13' 'product130' 'product131' 'product132'
 'product133' 'product134' 'product135' 'product136' 'product137'
 'product138' 'product14' 'product140' 'product141' 'product142'
 'product143' 'product144' 'product145' 'product146' 'product147'
 'product148' 'product149' 'product15' 'product150' 'product151'
 'product152' 'product153' 'product155' 'product156' 'product157'
 'product16' 'product17' 'product19' 'product2' 'product20' 'product24'
 'product26' 'product27' 'product28' 'product29' 'product3' 'product30'
 'product31' 'product32' 'product33' 'product34' 'product35' 'product3

In [3]:
print("train中缺失值：\n",train.isnull().sum())
print("\ntime中缺失值：\n",time.isnull().sum())
print("\nprofit_rate中缺失值：\n",profit_rate.isnull().sum())

train中缺失值：
 product_pid             0
transaction_date        0
apply_amt               0
redeem_amt              0
net_in_amt              0
uv_fundown              0
uv_stableown            0
uv_fundopt              0
uv_fundmarket           0
uv_termmarket           0
during_days           174
total_net_value     28160
dtype: int64

time中缺失值：
 stat_date          0
is_trade           0
next_trade_date    0
last_trade_date    0
is_week_end        0
is_month_end       0
is_quarter_end     0
is_year_end        0
trade_day_rank     0
dtype: int64

profit_rate中缺失值：
 enddate    0
yield      0
dtype: int64


In [4]:
train['during_days'] = train.groupby('product_pid')['during_days'].transform(lambda x:  x.ffill().bfill())
print("train中缺失值：\n",train.isnull().sum())

train中缺失值：
 product_pid             0
transaction_date        0
apply_amt               0
redeem_amt              0
net_in_amt              0
uv_fundown              0
uv_stableown            0
uv_fundopt              0
uv_fundmarket           0
uv_termmarket           0
during_days             0
total_net_value     28160
dtype: int64


In [5]:
train['total_net_value'] = train['total_net_value'].fillna(
    train.groupby('product_pid')['net_in_amt'].transform('sum')
)

In [6]:
print("train中缺失值：\n",train.isnull().sum())

train中缺失值：
 product_pid         0
transaction_date    0
apply_amt           0
redeem_amt          0
net_in_amt          0
uv_fundown          0
uv_stableown        0
uv_fundopt          0
uv_fundmarket       0
uv_termmarket       0
during_days         0
total_net_value     0
dtype: int64


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43399 entries, 0 to 43398
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_pid       43399 non-null  object 
 1   transaction_date  43399 non-null  int64  
 2   apply_amt         43399 non-null  float64
 3   redeem_amt        43399 non-null  float64
 4   net_in_amt        43399 non-null  float64
 5   uv_fundown        43399 non-null  float64
 6   uv_stableown      43399 non-null  float64
 7   uv_fundopt        43399 non-null  float64
 8   uv_fundmarket     43399 non-null  float64
 9   uv_termmarket     43399 non-null  float64
 10  during_days       43399 non-null  float64
 11  total_net_value   43399 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 4.0+ MB


In [8]:
train = pd.merge(train, profit_rate[['enddate', 'yield']], 
                 left_on='transaction_date', right_on='enddate', 
                 how='left')
print(train.head())

  product_pid  transaction_date  apply_amt  redeem_amt  net_in_amt  \
0    product1          20210104   0.001642    0.000851    0.000791   
1    product1          20210105   0.000170    0.002983   -0.002813   
2    product1          20210106   0.002941    0.001342    0.001599   
3    product1          20210107   0.005226    0.000000    0.005226   
4    product1          20210108   0.002587    0.000000    0.002587   

   uv_fundown  uv_stableown  uv_fundopt  uv_fundmarket  uv_termmarket  \
0    791827.0           0.0   8203723.0            0.0            0.0   
1    595051.0           0.0   4432183.0            0.0            0.0   
2    759031.0           0.0   6924679.0            0.0            0.0   
3    299887.0           0.0   4399387.0            0.0            0.0   
4    267091.0           0.0   2333239.0            0.0            0.0   

   during_days  total_net_value   enddate   yield  
0          0.0        13.232167  20210104  0.0283  
1          0.0        13.232167  202

In [9]:
train = train.drop(columns=['enddate'])
print(train.head())

  product_pid  transaction_date  apply_amt  redeem_amt  net_in_amt  \
0    product1          20210104   0.001642    0.000851    0.000791   
1    product1          20210105   0.000170    0.002983   -0.002813   
2    product1          20210106   0.002941    0.001342    0.001599   
3    product1          20210107   0.005226    0.000000    0.005226   
4    product1          20210108   0.002587    0.000000    0.002587   

   uv_fundown  uv_stableown  uv_fundopt  uv_fundmarket  uv_termmarket  \
0    791827.0           0.0   8203723.0            0.0            0.0   
1    595051.0           0.0   4432183.0            0.0            0.0   
2    759031.0           0.0   6924679.0            0.0            0.0   
3    299887.0           0.0   4399387.0            0.0            0.0   
4    267091.0           0.0   2333239.0            0.0            0.0   

   during_days  total_net_value   yield  
0          0.0        13.232167  0.0283  
1          0.0        13.232167  0.0284  
2          0.0

In [10]:
time=time.drop(columns=['next_trade_date','last_trade_date'])
time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791 entries, 0 to 790
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   stat_date       791 non-null    int64
 1   is_trade        791 non-null    int64
 2   is_week_end     791 non-null    int64
 3   is_month_end    791 non-null    int64
 4   is_quarter_end  791 non-null    int64
 5   is_year_end     791 non-null    int64
 6   trade_day_rank  791 non-null    int64
dtypes: int64(7)
memory usage: 43.4 KB


In [11]:
train = pd.merge(train, time, left_on='transaction_date', right_on='stat_date', how='left')
print(train.head())

  product_pid  transaction_date  apply_amt  redeem_amt  net_in_amt  \
0    product1          20210104   0.001642    0.000851    0.000791   
1    product1          20210105   0.000170    0.002983   -0.002813   
2    product1          20210106   0.002941    0.001342    0.001599   
3    product1          20210107   0.005226    0.000000    0.005226   
4    product1          20210108   0.002587    0.000000    0.002587   

   uv_fundown  uv_stableown  uv_fundopt  uv_fundmarket  uv_termmarket  \
0    791827.0           0.0   8203723.0            0.0            0.0   
1    595051.0           0.0   4432183.0            0.0            0.0   
2    759031.0           0.0   6924679.0            0.0            0.0   
3    299887.0           0.0   4399387.0            0.0            0.0   
4    267091.0           0.0   2333239.0            0.0            0.0   

   during_days  total_net_value   yield  stat_date  is_trade  is_week_end  \
0          0.0        13.232167  0.0283   20210104         1   

In [12]:
train=train.drop(columns=['stat_date'])

In [13]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43399 entries, 0 to 43398
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_pid       43399 non-null  object 
 1   transaction_date  43399 non-null  int64  
 2   apply_amt         43399 non-null  float64
 3   redeem_amt        43399 non-null  float64
 4   net_in_amt        43399 non-null  float64
 5   uv_fundown        43399 non-null  float64
 6   uv_stableown      43399 non-null  float64
 7   uv_fundopt        43399 non-null  float64
 8   uv_fundmarket     43399 non-null  float64
 9   uv_termmarket     43399 non-null  float64
 10  during_days       43399 non-null  float64
 11  total_net_value   43399 non-null  float64
 12  yield             43399 non-null  float64
 13  is_trade          43399 non-null  int64  
 14  is_week_end       43399 non-null  int64  
 15  is_month_end      43399 non-null  int64  
 16  is_quarter_end    43399 non-null  int64 

In [14]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings('ignore')

def create_features(df, is_train=True):
    """创建特征，区分训练集和测试集"""
    df = df.copy()
    df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%Y%m%d')
    df['year'] = df['transaction_date'].dt.year
    df['month'] = df['transaction_date'].dt.month
    df['day'] = df['transaction_date'].dt.day
    df['dayofweek'] = df['transaction_date'].dt.dayofweek
    
    if is_train:
        for lag in [1, 3, 7, 14, 30]:
            df[f'apply_amt_lag_{lag}'] = df.groupby('product_pid')['apply_amt'].shift(lag)
            df[f'redeem_amt_lag_{lag}'] = df.groupby('product_pid')['redeem_amt'].shift(lag)
        
        for window in [7, 14, 30]:
            df[f'apply_amt_mean_{window}d'] = df.groupby('product_pid')['apply_amt'].rolling(window=window).mean().reset_index(0, drop=True)
            df[f'redeem_amt_mean_{window}d'] = df.groupby('product_pid')['redeem_amt'].rolling(window=window).mean().reset_index(0, drop=True)
            df[f'apply_amt_std_{window}d'] = df.groupby('product_pid')['apply_amt'].rolling(window=window).std().reset_index(0, drop=True)
            df[f'redeem_amt_std_{window}d'] = df.groupby('product_pid')['redeem_amt'].rolling(window=window).std().reset_index(0, drop=True)
    
    return df

def evaluate_model(y_true, y_pred):
    """使用归一化的MSE作为评分"""
    try:
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        score = 1 / (1 + mse)  # MSE越小，score越接近1
        return {
            'mse': mse,
            'mae': mae,
            'r2': r2,
            'score': score
        }
    except:
        return {
            'mse': float('inf'),
            'mae': float('inf'),
            'r2': float('-inf'),
            'score': 0
        }

def simple_prediction(train_df, test_df):
    """对于数据较少的产品使用简单预测方法"""
    results = {}
    results['apply_amt_pred'] = np.full(len(test_df), train_df['apply_amt'].mean())
    results['redeem_amt_pred'] = np.full(len(test_df), train_df['redeem_amt'].mean())
    results['net_in_amt_pred'] = results['apply_amt_pred'] - results['redeem_amt_pred']
    
    return {
        'predictions': results,
        'best_models': {'apply': 'mean', 'redeem': 'mean'},
        'best_scores': {
            'apply': evaluate_model(train_df['apply_amt'], np.full(len(train_df), train_df['apply_amt'].mean())),
            'redeem': evaluate_model(train_df['redeem_amt'], np.full(len(train_df), train_df['redeem_amt'].mean()))
        }
    }
def predict_product_amounts(train_df, test_df, product_id):
    """对单个产品进行时间序列预测"""
    product_train = train_df[train_df['product_pid'] == product_id].sort_values('transaction_date')
    product_test = test_df[test_df['product_pid'] == product_id].sort_values('transaction_date')
    
    if len(product_train) < 30:
        return simple_prediction(product_train, product_test)
    
    results = {}
    best_models = {'apply': None, 'redeem': None}
    best_scores = {'apply': {'score': 0}, 'redeem': {'score': 0}}
    best_predictions = {'apply': None, 'redeem': None}
    
    product_train = create_features(product_train, is_train=True)
    product_test = create_features(product_test, is_train=False)
    
    base_features = ['year', 'month', 'day', 'dayofweek']
    train_features = base_features.copy()
    
    for col in product_train.columns:
        if 'lag' in col or 'mean' in col or 'std' in col:
            train_features.append(col)
    
    product_train = product_train.fillna(0)
    product_test = product_test.fillna(0)
    
    for target in ['apply_amt', 'redeem_amt']:
        target_type = target.split('_')[0]
        model_scores = {}  # 存储每个模型的得分
        model_predictions = {}  # 存储每个模型的预测结果
        model_full_scores = {}  # 存储每个模型的完整评分信息
        
        # 1. ARIMA模型
        try:
            best_arima_score = 0
            best_arima_param = None
            best_arima_pred = None
            best_arima_full_score = None
            
            for order in [(1,1,1), (2,1,2), (1,1,2), (2,1,1)]:
                try:
                    model = ARIMA(product_train[target], order=order)
                    model_fit = model.fit()
                    pred = model_fit.forecast(steps=len(product_test))
                    score_info = evaluate_model(product_train[target].iloc[-len(pred):], pred)
                    if score_info['score'] > best_arima_score:
                        best_arima_score = score_info['score']
                        best_arima_param = order
                        best_arima_pred = pred
                        best_arima_full_score = score_info
                except:
                    continue
            
            if best_arima_pred is not None:
                model_scores['ARIMA'] = best_arima_score
                model_predictions['ARIMA'] = best_arima_pred
                model_full_scores['ARIMA'] = best_arima_full_score
        except:
            pass
        
        # 2. RandomForest模型
        try:
            best_rf_score = 0
            best_rf_param = None
            best_rf_pred = None
            best_rf_full_score = None
            
            for n_estimators in [100, 200]:
                for max_depth in [10, 15]:
                    model = RandomForestRegressor(n_estimators=n_estimators, 
                                                max_depth=max_depth, 
                                                random_state=42)
                    cv_scores = []
                    cv_full_scores = []
                    
                    tscv = TimeSeriesSplit(n_splits=3)
                    for train_idx, val_idx in tscv.split(product_train):
                        train_data = product_train.iloc[train_idx]
                        val_data = product_train.iloc[val_idx]
                        model.fit(train_data[train_features], train_data[target])
                        pred = model.predict(val_data[base_features])
                        score_info = evaluate_model(val_data[target], pred)
                        cv_scores.append(score_info['score'])
                        cv_full_scores.append(score_info)
                    
                    avg_score = np.mean(cv_scores)
                    if avg_score > best_rf_score:
                        best_rf_score = avg_score
                        best_rf_param = {'n_estimators': n_estimators, 'max_depth': max_depth}
                        model.fit(product_train[train_features], product_train[target])
                        best_rf_pred = model.predict(product_test[base_features])
                        best_rf_full_score = {
                            'score': avg_score,
                            'mse': np.mean([s['mse'] for s in cv_full_scores]),
                            'mae': np.mean([s['mae'] for s in cv_full_scores]),
                            'r2': np.mean([s['r2'] for s in cv_full_scores])
                        }
            
            if best_rf_pred is not None:
                model_scores['RandomForest'] = best_rf_score
                model_predictions['RandomForest'] = best_rf_pred
                model_full_scores['RandomForest'] = best_rf_full_score
        except:
            pass
        
        # 3. XGBoost模型
        try:
            best_xgb_score = 0
            best_xgb_param = None
            best_xgb_pred = None
            best_xgb_full_score = None
            
            for n_estimators in [100, 200]:
                for max_depth in [6, 8]:
                    model = XGBRegressor(n_estimators=n_estimators, 
                                       max_depth=max_depth, 
                                       random_state=42)
                    cv_scores = []
                    cv_full_scores = []
                    
                    tscv = TimeSeriesSplit(n_splits=3)
                    for train_idx, val_idx in tscv.split(product_train):
                        train_data = product_train.iloc[train_idx]
                        val_data = product_train.iloc[val_idx]
                        model.fit(train_data[train_features], train_data[target])
                        pred = model.predict(val_data[base_features])
                        score_info = evaluate_model(val_data[target], pred)
                        cv_scores.append(score_info['score'])
                        cv_full_scores.append(score_info)
                    
                    avg_score = np.mean(cv_scores)
                    if avg_score > best_xgb_score:
                        best_xgb_score = avg_score
                        best_xgb_param = {'n_estimators': n_estimators, 'max_depth': max_depth}
                        model.fit(product_train[train_features], product_train[target])
                        best_xgb_pred = model.predict(product_test[base_features])
                        best_xgb_full_score = {
                            'score': avg_score,
                            'mse': np.mean([s['mse'] for s in cv_full_scores]),
                            'mae': np.mean([s['mae'] for s in cv_full_scores]),
                            'r2': np.mean([s['r2'] for s in cv_full_scores])
                        }
            
            if best_xgb_pred is not None:
                model_scores['XGBoost'] = best_xgb_score
                model_predictions['XGBoost'] = best_xgb_pred
                model_full_scores['XGBoost'] = best_xgb_full_score
        except:
            pass
        
        # 4. GradientBoosting模型
        try:
            best_gb_score = 0
            best_gb_param = None
            best_gb_pred = None
            best_gb_full_score = None
            
            for n_estimators in [100, 200]:
                for max_depth in [5, 7]:
                    model = GradientBoostingRegressor(n_estimators=n_estimators, 
                                                    max_depth=max_depth, 
                                                    random_state=42)
                    cv_scores = []
                    cv_full_scores = []
                    
                    tscv = TimeSeriesSplit(n_splits=3)
                    for train_idx, val_idx in tscv.split(product_train):
                        train_data = product_train.iloc[train_idx]
                        val_data = product_train.iloc[val_idx]
                        model.fit(train_data[train_features], train_data[target])
                        pred = model.predict(val_data[base_features])
                        score_info = evaluate_model(val_data[target], pred)
                        cv_scores.append(score_info['score'])
                        cv_full_scores.append(score_info)
                    
                    avg_score = np.mean(cv_scores)
                    if avg_score > best_gb_score:
                        best_gb_score = avg_score
                        best_gb_param = {'n_estimators': n_estimators, 'max_depth': max_depth}
                        model.fit(product_train[train_features], product_train[target])
                        best_gb_pred = model.predict(product_test[base_features])
                        best_gb_full_score = {
                            'score': avg_score,
                            'mse': np.mean([s['mse'] for s in cv_full_scores]),
                            'mae': np.mean([s['mae'] for s in cv_full_scores]),
                            'r2': np.mean([s['r2'] for s in cv_full_scores])
                        }
            
            if best_gb_pred is not None:
                model_scores['GradientBoosting'] = best_gb_score
                model_predictions['GradientBoosting'] = best_gb_pred
                model_full_scores['GradientBoosting'] = best_gb_full_score
        except:
            pass
        
        # 选择最佳模型
        if model_scores:
            best_model = max(model_scores.items(), key=lambda x: x[1])[0]
            best_models[target_type] = best_model
            best_scores[target_type] = model_full_scores[best_model]
            if target == 'apply_amt':
                results['apply_amt_pred'] = model_predictions[best_model]
            else:
                results['redeem_amt_pred'] = model_predictions[best_model]
            
            print(f"\n{target} 各模型得分:")
            for model_name, score in model_scores.items():
                print(f"{model_name}: {score:.4f}")
    
    if 'apply_amt_pred' in results and 'redeem_amt_pred' in results:
        results['net_in_amt_pred'] = results['apply_amt_pred'] - results['redeem_amt_pred']
        return {
            'predictions': results,
            'best_models': best_models,
            'best_scores': best_scores
        }
    else:
        return simple_prediction(product_train, product_test)

def predict_all_products(train_df, test_df):
    """预测所有产品的金额"""
    unique_products = test_df['product_pid'].unique()
    results_df = test_df.copy()
    model_statistics = {'apply': {}, 'redeem': {}}
    
    # 初始化预测列
    results_df['apply_amt_pred'] = 0.0
    results_df['redeem_amt_pred'] = 0.0
    results_df['net_in_amt_pred'] = 0.0
    
    for product_id in unique_products:
        print(f"正在处理产品: {product_id}")
        result = predict_product_amounts(train_df, test_df, product_id)
        
        if result is not None:
            mask = results_df['product_pid'] == product_id
            
            # 确保预测值是数值类型
            apply_pred = np.array(result['predictions']['apply_amt_pred']).astype(float)
            redeem_pred = np.array(result['predictions']['redeem_amt_pred']).astype(float)
            net_pred = np.array(result['predictions']['net_in_amt_pred']).astype(float)
            
            # 检查预测值的长度是否匹配
            expected_length = mask.sum()
            if len(apply_pred) != expected_length:
                print(f"警告：产品 {product_id} 的预测长度不匹配")
                continue
                
            # 保存预测结果
            results_df.loc[mask, 'apply_amt_pred'] = apply_pred
            results_df.loc[mask, 'redeem_amt_pred'] = redeem_pred
            results_df.loc[mask, 'net_in_amt_pred'] = net_pred
            
            # 打印一些验证信息
            print(f"已保存预测值 - 数量: {len(apply_pred)}, 申购均值: {np.mean(apply_pred):.2f}, 赎回均值: {np.mean(redeem_pred):.2f}")
            
            for target in ['apply', 'redeem']:
                model_name = result['best_models'][target]
                model_statistics[target][model_name] = model_statistics[target].get(model_name, 0) + 1
            
            print(f"最佳模型 - 申购: {result['best_models']['apply']}, 赎回: {result['best_models']['redeem']}")
            print(f"模型评分 - 申购: score={result['best_scores']['apply']['score']:.4f}")
            print(f"模型评分 - 赎回: score={result['best_scores']['redeem']['score']:.4f}")
            print("-------------------")
    
    # 验证预测结果
    print("\n预测结果统计:")
    print("申购预测 - 均值:", results_df['apply_amt_pred'].mean())
    print("申购预测 - 最小值:", results_df['apply_amt_pred'].min())
    print("申购预测 - 最大值:", results_df['apply_amt_pred'].max())
    print("赎回预测 - 均值:", results_df['redeem_amt_pred'].mean())
    print("赎回预测 - 最小值:", results_df['redeem_amt_pred'].min())
    print("赎回预测 - 最大值:", results_df['redeem_amt_pred'].max())
    
    print("\n模型使用统计:")
    for target, stats in model_statistics.items():
        print(f"\n{target}预测最佳模型统计:")
        for model_name, count in stats.items():
            print(f"{model_name}: {count}次")
    
    return results_df

# 开始预测
print("开始预测...")
results = predict_all_products(train, test)

# 验证并保存结果
print("\n保存前的最终检查:")
print("预测列的非空值数量:")
print("apply_amt_pred:", results['apply_amt_pred'].count())
print("redeem_amt_pred:", results['redeem_amt_pred'].count())
print("net_in_amt_pred:", results['net_in_amt_pred'].count())

# 检查是否有空值或异常值
print("\n检查是否有空值或0值:")
print("apply_amt_pred 为0的数量:", (results['apply_amt_pred'] == 0).sum())
print("redeem_amt_pred 为0的数量:", (results['redeem_amt_pred'] == 0).sum())
print("net_in_amt_pred 为0的数量:", (results['net_in_amt_pred'] == 0).sum())

# 保存结果
print("\n正在保存结果...")
try:
    # 确保预测值是数值类型
    results['apply_amt_pred'] = results['apply_amt_pred'].astype(float)
    results['redeem_amt_pred'] = results['redeem_amt_pred'].astype(float)
    results['net_in_amt_pred'] = results['net_in_amt_pred'].astype(float)
    
    # 保存文件
    save_path = 'D:\\天池比赛\\基金预测\\predict_table.csv'
    results.to_csv(save_path, index=False)
    print(f"预测完成，结果已保存到：{save_path}")
    
    # 验证保存的文件
    saved_results = pd.read_csv(save_path)
    print("\n保存后的验证:")
    print("文件中的预测值统计:")
    print("apply_amt_pred 均值:", saved_results['apply_amt_pred'].mean())
    print("redeem_amt_pred 均值:", saved_results['redeem_amt_pred'].mean())
    print("net_in_amt_pred 均值:", saved_results['net_in_amt_pred'].mean())
    
except Exception as e:
    print(f"保存文件时出错：{str(e)}")
    print("请检查文件是否被其他程序占用，或尝试使用其他文件名保存")

开始预测...
正在处理产品: product1

apply_amt 各模型得分:
ARIMA: 0.2020

redeem_amt 各模型得分:
ARIMA: 0.9994
已保存预测值 - 数量: 10, 申购均值: 0.84, 赎回均值: 0.02
最佳模型 - 申购: ARIMA, 赎回: ARIMA
模型评分 - 申购: score=0.2020
模型评分 - 赎回: score=0.9994
-------------------
正在处理产品: product100

apply_amt 各模型得分:
ARIMA: 0.9969

redeem_amt 各模型得分:
ARIMA: 0.9585
已保存预测值 - 数量: 10, 申购均值: 0.18, 赎回均值: 0.35
最佳模型 - 申购: ARIMA, 赎回: ARIMA
模型评分 - 申购: score=0.9969
模型评分 - 赎回: score=0.9585
-------------------
正在处理产品: product101

apply_amt 各模型得分:
ARIMA: 0.1254

redeem_amt 各模型得分:
ARIMA: 0.0279
已保存预测值 - 数量: 10, 申购均值: 3.55, 赎回均值: 5.61
最佳模型 - 申购: ARIMA, 赎回: ARIMA
模型评分 - 申购: score=0.1254
模型评分 - 赎回: score=0.0279
-------------------
正在处理产品: product102

apply_amt 各模型得分:
ARIMA: 0.9952

redeem_amt 各模型得分:
ARIMA: 0.6400
已保存预测值 - 数量: 10, 申购均值: 0.10, 赎回均值: 0.58
最佳模型 - 申购: ARIMA, 赎回: ARIMA
模型评分 - 申购: score=0.9952
模型评分 - 赎回: score=0.6400
-------------------
正在处理产品: product103

apply_amt 各模型得分:
ARIMA: 0.1052

redeem_amt 各模型得分:
ARIMA: 0.2633
已保存预测值 - 数量: 10, 申购均值: 2.70, 赎